In [1]:
import pandas as pd
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 10 min

In [2]:
df = pd.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
df.methods = df.methods.replace("['svm']", "1c")
df.methods = df.methods.replace("['dt']", "1c")
df.methods = df.methods.replace("['rf']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df = df_binary
df10 = df[df.time_budget==10]
df30 = df[df.time_budget==30]
df60 = df[df.time_budget==60]


df.head(5)

,dataset,time_budget,methods,f1score
0,vowel,10,fc,1.000000
1,openml_phpJNxH0q,10,fc,0.976378
2,dataset_31_credit-g,10,fc,0.830769
5,audiology,10,fc,0.882353
6,arrhythmia,10,fc,0.666667


In [3]:
rp.summary_cont(df10['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,161.0,0.823724,0.197346,0.015553,0.793008,0.85444


In [4]:
rp.summary_cont(df30['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,223.0,0.828943,0.192694,0.012904,0.803514,0.854373


In [5]:
rp.summary_cont(df60['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,197.0,0.803375,0.233856,0.016662,0.770516,0.836233


In [15]:
stats.f_oneway(df10['f1score'][df10['methods'] == '1c'], 
             df10['f1score'][df10['methods'] == '3c'],
             df10['f1score'][df10['methods'] == 'def'],
             df10['f1score'][df10['methods'] == 'fc'])

F_onewayResult(statistic=2.4775141122193687, pvalue=0.06334924641031792)

In [16]:
stats.f_oneway(df30['f1score'][df30['methods'] == '1c'], 
             df30['f1score'][df30['methods'] == '3c'],
             df30['f1score'][df30['methods'] == 'def'],
             df30['f1score'][df30['methods'] == 'fc'])

F_onewayResult(statistic=0.9124212072975914, pvalue=0.4357441943809103)

In [17]:
stats.f_oneway(df60['f1score'][df60['methods'] == '1c'], 
             df60['f1score'][df60['methods'] == '3c'],
             df60['f1score'][df60['methods'] == 'def'],
             df60['f1score'][df60['methods'] == 'fc'])

F_onewayResult(statistic=0.16797382316628268, pvalue=0.9178872637179489)

In [18]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model10 = ols('f1score ~ C(methods)', df10).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model10.df_model: .0f},{model10.df_resid: .0f}) = {model10.fvalue: .3f}, p = {model10.f_pvalue: .4f}")
    model10.summary()

Overall model F( 3, 157) =  2.478, p =  0.0633


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     2.478
Date:                Fri, 01 Nov 2019   Prob (F-statistic):             0.0633
Time:                        17:24:24   Log-Likelihood:                 37.046
No. Observations:                 161   AIC:                            -66.09
Df Residuals:                     157   BIC:                            -53.77
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7840      0.021     36.692      0.000       0.742       0.826
C(methods)[T.3c]      0.0533      0.060      0.887      0.376      -0.065       0.172
C(methods)[T.def]     0.0887      0.048      1.828      0.069      -0.007       0.184
C(methods)[T.fc]      0.0865      0.036      2.417      0.017       0.016       0.157
==============================================================================
Omnibus:                       49.648   Durbin-Watson:                   2.261
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.222
Skew:                          -1.542   Prob(JB):                     1.15e-19
Kurtosis:                       4.869   Cond. No.                         4.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model30 = ols('f1score ~ C(methods)', df30).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model30.df_model: .0f},{model30.df_resid: .0f}) = {model30.fvalue: .3f}, p = {model30.f_pvalue: .4f}")
    model30.summary()

Overall model F( 3, 219) =  0.912, p =  0.4357


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9124
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.436
Time:                        17:24:25   Log-Likelihood:                 52.666
No. Observations:                 223   AIC:                            -97.33
Df Residuals:                     219   BIC:                            -83.70
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8200      0.017     47.929      0.000       0.786       0.854
C(methods)[T.3c]      0.0500      0.044      1.145      0.253      -0.036       0.136
C(methods)[T.def]     0.0536      0.045      1.181      0.239      -0.036       0.143
C(methods)[T.fc]     -0.0055      0.032     -0.173      0.863      -0.068       0.057
==============================================================================
Omnibus:                       70.540   Durbin-Watson:                   2.290
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              135.329
Skew:                          -1.632   Prob(JB):                     4.11e-30
Kurtosis:                       4.976   Cond. No.                         4.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model60 = ols('f1score ~ C(methods)', df60).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model60.df_model: .0f},{model60.df_resid: .0f}) = {model60.fvalue: .3f}, p = {model60.f_pvalue: .4f}")
    model10.summary()

Overall model F( 3, 193) =  0.168, p =  0.9179


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     2.478
Date:                Fri, 01 Nov 2019   Prob (F-statistic):             0.0633
Time:                        17:24:31   Log-Likelihood:                 37.046
No. Observations:                 161   AIC:                            -66.09
Df Residuals:                     157   BIC:                            -53.77
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7840      0.021     36.692      0.000       0.742       0.826
C(methods)[T.3c]      0.0533      0.060      0.887      0.376      -0.065       0.172
C(methods)[T.def]     0.0887      0.048      1.828      0.069      -0.007       0.184
C(methods)[T.fc]      0.0865      0.036      2.417      0.017       0.016       0.157
==============================================================================
Omnibus:                       49.648   Durbin-Watson:                   2.261
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.222
Skew:                          -1.542   Prob(JB):                     1.15e-19
Kurtosis:                       4.869   Cond. No.                         4.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df10['f1score'], df10['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0533 0.7878 -0.1028 0.2095  False
    1c    def   0.0887  0.264 -0.0373 0.2146  False
    1c     fc   0.0865 0.0782 -0.0064 0.1794  False
    3c    def   0.0353    0.9 -0.1493 0.2199  False
    3c     fc   0.0331    0.9 -0.1307  0.197  False
   def     fc  -0.0022    0.9 -0.1376 0.1332  False
---------------------------------------------------


In [22]:
mc = MultiComparison(df30['f1score'], df30['methods'])
mc_results = mc.tukeyhsd(model30.f_pvalue)
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.44
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c     0.05 0.6431 -0.0159 0.1159  False
    1c    def   0.0536 0.6232 -0.0149 0.1221  False
    1c     fc  -0.0055    0.9 -0.0534 0.0424  False
    3c    def   0.0036    0.9 -0.0842 0.0914  False
    3c     fc  -0.0555 0.6405 -0.1283 0.0173  False
   def     fc  -0.0591 0.6205 -0.1343 0.0161  False
---------------------------------------------------


In [23]:
mc = MultiComparison(df60['f1score'], df60['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    1c     3c   0.0272   0.9 -0.1022 0.1567  False
    1c    def   0.0272   0.9 -0.1005 0.1549  False
    1c     fc   0.0133   0.9 -0.1081 0.1348  False
    3c    def  -0.0001   0.9 -0.1617 0.1615  False
    3c     fc  -0.0139   0.9 -0.1706 0.1428  False
   def     fc  -0.0138   0.9 -0.1691 0.1414  False
--------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C 10 min

In [ ]:
df = pd.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svc")
df.methods = df.methods.replace("['decision_tree']", "dt")
df.methods = df.methods.replace("['random_forest']", "rf")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df = df_binary
df10 = df[df.time_budget==10]
df30 = df[df.time_budget==30]
df60 = df[df.time_budget==60]
df.shape

In [ ]:
rp.summary_cont(df10['f1score'])

In [ ]:
rp.summary_cont(df30['f1score'])

In [ ]:
rp.summary_cont(df60['f1score'])

In [ ]:
stats.f_oneway(df10['f1score'][df10['methods'] == '1c'], 
             df10['f1score'][df10['methods'] == '3c'],
             df10['f1score'][df10['methods'] == 'fc'])

In [ ]:
stats.f_oneway(df30['f1score'][df30['methods'] == '1c'], 
             df30['f1score'][df30['methods'] == '3c'],
             df30['f1score'][df30['methods'] == 'fc'])

In [ ]:
stats.f_oneway(df60['f1score'][df60['methods'] == '1c'], 
             df60['f1score'][df60['methods'] == '3c'],
             df60['f1score'][df60['methods'] == 'fc'])

In [ ]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model10 = ols('f1score ~ C(methods)', df10).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model10.df_model: .0f},{model10.df_resid: .0f}) = {model10.fvalue: .3f}, p = {model10.f_pvalue: .4f}")
    model10.summary()

In [ ]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model30 = ols('f1score ~ C(methods)', df30).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model30.df_model: .0f},{model30.df_resid: .0f}) = {model30.fvalue: .3f}, p = {model30.f_pvalue: .4f}")
    model30.summary()

In [ ]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model60 = ols('f1score ~ C(methods)', df60).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model60.df_model: .0f},{model60.df_resid: .0f}) = {model60.fvalue: .3f}, p = {model60.f_pvalue: .4f}")
    model10.summary()

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df10['f1score'], df10['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

In [ ]:
mc = MultiComparison(df30['f1score'], df30['methods'])
mc_results = mc.tukeyhsd(model30.f_pvalue)
print(mc_results)

In [ ]:
mc = MultiComparison(df60['f1score'], df60['methods'])
mc_results = mc.tukeyhsd(model60.f_pvalue)
print(mc_results)